### LoRA Train

In [ ]:
# -------------------------------------------------------------------
# 01 ROS2 Command Dataset ပြင်ဆင်ခြင်း
# -------------------------------------------------------------------
from datasets import Dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType


# -------------------------------------------------------------------
# 02 Dataset
# -------------------------------------------------------------------
data = {
    "instruction": [
        "Move robot forward at 0.5 m/s",
        "Turn robot left 90 degrees",
        "Stop the robot",
        "Navigate to position x=2, y=3",
        "Rotate robot clockwise"
    ],
    "output": [
        "ros2 topic pub /cmd_vel geometry_msgs/msg/Twist '{linear: {x: 0.5}}'",
        "ros2 topic pub /cmd_vel geometry_msgs/msg/Twist '{angular: {z: 1.57}}'",
        "ros2 topic pub /cmd_vel geometry_msgs/msg/Twist '{}'",
        "ros2 action send_goal /navigate_to_pose nav2_msgs/action/NavigateToPose '{pose: {pose: {position: {x: 2.0, y: 3.0}}}}'",
        "ros2 topic pub /cmd_vel geometry_msgs/msg/Twist '{angular: {z: -1.57}}'"
    ]
}

dataset = Dataset.from_dict(data)
dataset = dataset.train_test_split(test_size=0.2)
print(dataset)


# -------------------------------------------------------------------
# 03 Tokenizer & Tokenization
# -------------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-1.5B-Instruct")
tokenizer.pad_token = tokenizer.eos_token


def tokenize_function(examples):
    texts = [
        f"### Instruction:\n{inst}\n\n### Command:\n{out}"
        for inst, out in zip(examples["instruction"], examples["output"])
    ]

    tokenized = tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=256,
    )

    labels = []
    for ids in tokenized["input_ids"]:
        labels.append([
            token if token != tokenizer.pad_token_id else -100
            for token in ids
        ])

    tokenized["labels"] = labels
    return tokenized


tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

print(tokenized_dataset)


# -------------------------------------------------------------------
# 04 LoRA Configuration
# -------------------------------------------------------------------
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,

# Module,Dimension (Input × Output),တွက်ချက်ပုံ
# q_proj,   1536×1536,      dmodel​×dmodel​
# k_proj,   1536×1536,      dmodel​×dmodel​
# v_proj,   1536×1536,      dmodel​×dmodel​
# o_proj,    1536×1536,     dmodel​×dmodel​
# gate_proj, 1536×8960,     dmodel​×Intermediate Size
# up_proj,   1536×8960,     dmodel​×Intermediate Size
# down_proj, 8960×1536,     Intermediate Size×dmodel​

    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ],
)


# -------------------------------------------------------------------
# 05 Base Model → PEFT Model  
# -------------------------------------------------------------------

# Qwen2.5-1.5B ၏ အခြေခံကိန်းဂဏန်း (Configuration) များမှာ-
#     Hidden Size ($d_{model}$): 1536
#     Intermediate Size (MLP): 8960
#     Number of Heads: 12

base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Coder-1.5B-Instruct",
    device_map="auto"
)

peft_model = get_peft_model(base_model, lora_config)
peft_model.print_trainable_parameters()


# -------------------------------------------------------------------
# 06 Training
# -------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./ros2_lora_model",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,

    logging_strategy="steps",
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",

    fp16=True,
    report_to="none",
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

print("Starting training...")
trainer.train()

trainer.save_model("./ros2_command_model_final")
print("Training complete!")


2025-12-24 08:20:20.135830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766564420.316521      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766564420.369624      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766564420.820981      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766564420.821020      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766564420.821022      55 computation_placer.cc:177] computation placer alr

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 4
    })
    test: Dataset({
        features: ['instruction', 'output'],
        num_rows: 1
    })
})


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


trainable params: 9,232,384 || all params: 1,552,946,688 || trainable%: 0.5945
Starting training...


Epoch,Training Loss,Validation Loss
1,2.362800,1.662189
2,1.612900,1.511866
3,1.178800,1.370880
4,0.894600,1.267853
5,0.691500,1.203773
6,0.540600,1.203773
7,0.542800,1.168796
8,0.440700,1.152602
9,0.373600,1.149202
10,0.357200,1.150394


Training complete!


In [4]:
import shutil

# Zip the 'result' folder
shutil.make_archive("/kaggle/working/ros2_command_model_final", 'zip', "/kaggle/working/ros2_command_model_final")

shutil.make_archive("/kaggle/working/ros2_lora_model", 'zip', "/kaggle/working/ros2_lora_model")


'/kaggle/working/ros2_lora_model.zip'

### LoRA testing

In [11]:
from peft import PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
import torch


# -------------------------------------------------------------------
# Load tokenizer (MUST match training)
# -------------------------------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-Coder-1.5B-Instruct"
)
tokenizer.pad_token = tokenizer.eos_token


# -------------------------------------------------------------------
# Load base model
# -------------------------------------------------------------------
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-Coder-1.5B-Instruct",
    device_map="auto"
)


# -------------------------------------------------------------------
# Load LoRA weights
# -------------------------------------------------------------------
peft_model = PeftModel.from_pretrained(
    base_model,
    "./ros2_command_model_final"
)

peft_model.eval()  # ✅ important for inference


# -------------------------------------------------------------------
# Inference
# -------------------------------------------------------------------
prompt = """### Instruction:
Move robot forward 3 meters

### Command:
"""

inputs = tokenizer(prompt, return_tensors="pt").to(peft_model.device)

outputs = peft_model.generate(
    **inputs,
    max_new_tokens=50,   # ✅ correct
    temperature=0.7,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Move robot forward 3 meters

### Command:
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist '{linear: {x: 3.0}}' -1

### Result:
Robot moves forward 3 meters.

---

### Instruction:
Turn robot clockwise by 90 degrees
